<a href="https://colab.research.google.com/github/Daivar/Deep_Learning_Models/blob/main/Caltech101__RIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!rm -rf datasets

In [10]:
!mkdir -p datasets
# !pip install gdown # not necessary when running on google collab
!gdown https://drive.google.com/uc?id=137RyRjvTBkBiIfeYBNZBtViDHQ6_Ewsp --output datasets/caltech101.tar.gz
!tar -xvzf datasets/caltech101.tar.gz --directory datasets
!mv datasets/101_ObjectCategories datasets/caltech101
!rm -rf datasets/caltech101/BACKGROUND_Google

Srautinio perdavimo išvestis sutrumpinta iki kelių paskutinių eilučių (5000).
101_ObjectCategories/chair/image_0005.jpg
101_ObjectCategories/chair/image_0006.jpg
101_ObjectCategories/chair/image_0007.jpg
101_ObjectCategories/chair/image_0008.jpg
101_ObjectCategories/chair/image_0010.jpg
101_ObjectCategories/chair/image_0011.jpg
101_ObjectCategories/chair/image_0012.jpg
101_ObjectCategories/chair/image_0013.jpg
101_ObjectCategories/chair/image_0014.jpg
101_ObjectCategories/chair/image_0016.jpg
101_ObjectCategories/chair/image_0017.jpg
101_ObjectCategories/chair/image_0018.jpg
101_ObjectCategories/chair/image_0019.jpg
101_ObjectCategories/chair/image_0020.jpg
101_ObjectCategories/chair/image_0022.jpg
101_ObjectCategories/chair/image_0023.jpg
101_ObjectCategories/chair/image_0024.jpg
101_ObjectCategories/chair/image_0025.jpg
101_ObjectCategories/chair/image_0026.jpg
101_ObjectCategories/chair/image_0028.jpg
101_ObjectCategories/chair/image_0029.jpg
101_ObjectCategories/chair/image_0030.jp

In [11]:
!ls -laht ./datasets
!du -hcs ./datasets/caltech101

total 126M
drwxr-xr-x 103  411  300 4.0K Jan  2 09:53 caltech101
drwxr-xr-x   3 root root 4.0K Jan  2 09:53 .
-rw-r--r--   1 root root 126M Jan  2 09:53 caltech101.tar.gz
drwxr-xr-x   1 root root 4.0K Jan  2 09:53 ..
133M	./datasets/caltech101
133M	total


In [12]:
import os
print(os.environ)

# finally the check
if 'COLAB_GPU' in os.environ:
   print("I'm running on Colab")

# you can also convert a python object into a dict using vars functions.
# ... the benefit being that it will be displayed line by line
vars(os.environ)['_data']

environ({'NO_GCE_CHECK': 'True', 'GCS_READ_CACHE_BLOCK_SIZE_MB': '16', 'CLOUDSDK_CONFIG': '/content/.config', 'CUDA_VERSION': '11.1.1', 'PATH': '/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin:/opt/bin', 'HOME': '/root', 'LD_LIBRARY_PATH': '/usr/local/nvidia/lib:/usr/local/nvidia/lib64', 'LANG': 'en_US.UTF-8', 'SHELL': '/bin/bash', 'LIBRARY_PATH': '/usr/local/cuda/lib64/stubs', 'SHLVL': '0', 'GCE_METADATA_TIMEOUT': '0', 'NCCL_VERSION': '2.7.8', 'NVIDIA_VISIBLE_DEVICES': 'all', 'DEBIAN_FRONTEND': 'noninteractive', 'CUDNN_VERSION': '8.0.5.39', 'LAST_FORCED_REBUILD': '20211202', 'JPY_PARENT_PID': '45', 'PYTHONPATH': '/env/python', 'DATALAB_SETTINGS_OVERRIDES': '{"kernelManagerProxyPort":6000,"kernelManagerProxyHost":"172.28.0.3","jupyterArgs":["--ip=\\"172.28.0.2\\""],"debugAdapterMultiplexerPath":"/usr/local/bin/dap_multiplexer","enableLsp":true}', 'ENV': '/root/.bashrc', 'GLIBCXX_FORCE_NEW

{b'CLICOLOR': b'1',
 b'CLOUDSDK_CONFIG': b'/content/.config',
 b'CLOUDSDK_PYTHON': b'python3',
 b'COLAB_GPU': b'0',
 b'CUDA_VERSION': b'11.1.1',
 b'CUDNN_VERSION': b'8.0.5.39',
 b'DATALAB_SETTINGS_OVERRIDES': b'{"kernelManagerProxyPort":6000,"kernelManagerProxyHost":"172.28.0.3","jupyterArgs":["--ip=\\"172.28.0.2\\""],"debugAdapterMultiplexerPath":"/usr/local/bin/dap_multiplexer","enableLsp":true}',
 b'DEBIAN_FRONTEND': b'noninteractive',
 b'ENABLE_DIRECTORYPREFETCHER': b'1',
 b'ENV': b'/root/.bashrc',
 b'GCE_METADATA_TIMEOUT': b'0',
 b'GCS_READ_CACHE_BLOCK_SIZE_MB': b'16',
 b'GIT_PAGER': b'cat',
 b'GLIBCPP_FORCE_NEW': b'1',
 b'GLIBCXX_FORCE_NEW': b'1',
 b'HOME': b'/root',
 b'HOSTNAME': b'ea6b3bd775f2',
 b'JPY_PARENT_PID': b'45',
 b'LANG': b'en_US.UTF-8',
 b'LAST_FORCED_REBUILD': b'20211202',
 b'LD_LIBRARY_PATH': b'/usr/local/nvidia/lib:/usr/local/nvidia/lib64',
 b'LD_PRELOAD': b'/usr/lib/x86_64-linux-gnu/libtcmalloc.so.4',
 b'LIBRARY_PATH': b'/usr/local/cuda/lib64/stubs',
 b'MPLBACKEN

In [13]:
import numpy as np
from numpy.linalg import norm
import pickle
from tqdm import tqdm, tqdm_notebook
import os
import random
import time
import math
import tensorflow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D

In [14]:
def model_picker(name):
    if (name == 'vgg16'):
        model = VGG16(weights='imagenet',
                      include_top=False,
                      input_shape=(224, 224, 3),
                      pooling='max')
    elif (name == 'vgg19'):
        model = VGG19(weights='imagenet',
                      include_top=False,
                      input_shape=(224, 224, 3),
                      pooling='max')
    elif (name == 'mobilenet'):
        model = MobileNet(weights='imagenet',
                          include_top=False,
                          input_shape=(224, 224, 3),
                          pooling='max',
                          depth_multiplier=1, # TODO :: explanation
                          alpha=1) # TODO :: explnation
    elif (name == 'inception'):
        model = InceptionV3(weights='imagenet',
                            include_top=False,
                            input_shape=(224, 224, 3),
                            pooling='max')
    elif (name == 'resnet'):
        model = ResNet50(weights='imagenet',
                         include_top=False,
                         input_shape=(224, 224, 3),
                         pooling='max')
    elif (name == 'xception'):
        model = Xception(weights='imagenet',
                         include_top=False,
                         input_shape=(224, 224, 3),
                         pooling='max')
    else:
        print("Specified model not available")
    return model

In [16]:
model_architecture = 'resnet'
model = model_picker(model_architecture)

In [17]:
def extract_features(img_path, model):
    input_shape = (224, 224, 3)
    img = image.load_img(img_path, target_size=(input_shape[0], input_shape[1]))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = tensorflow.keras.applications.resnet50.preprocess_input(expanded_img_array)

    features = model.predict(preprocessed_img)              # pass the image through CNN (no head!)
    flattened_features = features.flatten()
    normalized_features = flattened_features / norm(flattened_features) # comes from numpy.linalg
    return normalized_features

In [18]:
features = extract_features('datasets/caltech101/cougar_body/image_0025.jpg', model)
print(len(features))
print(features.shape) # flat

2048
(2048,)


In [ ]:
model.summary()

In [19]:
arr = np.array([30,2,1]) #linalg.norm - normalizuoja

print(arr)
print(norm(arr))
print(arr / norm(arr)) # Frobenius Norm / L2 norm - Linear Algebra. Between 0 and 1.

# Statistical normalization
print((arr - arr.mean()) / arr.std()) # centered around zero.

# you can also use sklearn for that
import numpy as np
from sklearn.preprocessing import normalize
norm1 = arr / np.linalg.norm(arr)
norm2 = normalize(arr[:,np.newaxis], axis=0).ravel()
print(np.all(norm1 == norm2))

[30  2  1]
30.083217912982647
[0.99723374 0.06648225 0.03324112]
[ 1.4135611  -0.66958157 -0.74397953]
True


In [20]:
model = model_picker('mobilenet')
%timeit -n 10 features = extract_features('datasets/caltech101/cougar_body/image_0025.jpg', model)

17235968/17225924 [==============================] - 0s 0us/step
10 loops, best of 5: 88.9 ms per loop


In [ ]:
# model = model_picker('vgg16')
# %timeit -n 10 features = extract_features('datasets/caltech101/cougar_body/image_0025.jpg', model)

# model = model_picker('vgg19')
# %timeit -n 10 features = extract_features('datasets/caltech101/cougar_body/image_0025.jpg', model)

# model = model_picker('mobilenet')
# %timeit -n 10 features = extract_features('datasets/caltech101/cougar_body/image_0025.jpg', model)

# model = model_picker('inception')
# %timeit -n 10 features = extract_features('datasets/caltech101/cougar_body/image_0025.jpg', model)

# model = model_picker('resnet')
# %timeit -n 10 features = extract_features('datasets/caltech101/cougar_body/image_0025.jpg', model)

# model = model_picker('xception')
# %timeit -n 10 features = extract_features('datasets/caltech101/cougar_body/image_0025.jpg', model)


# 2021
# mobilenet - 10 loops, best of 5: 62.5 ms per loop
# xception - 10 loops, best of 5: 72.8 ms per loopĘ

In [21]:
extensions = ['.jpg', '.JPG', '.jpeg', '.JPEG', '.png', '.PNG']

def get_file_list(root_dir):
    file_list = []
    for root, directories, filenames in os.walk(root_dir):
        for filename in filenames:
            if any(ext in filename for ext in extensions):
                file_list.append(os.path.join(root, filename))
    return file_list

In [22]:
from tqdm.notebook import tqdm as tqdm_nb

# path to the your datasets
root_dir = 'datasets/caltech101'
filenames = sorted(get_file_list(root_dir)[:1000]) # remove :1000 on final run

feature_list = []
model = model_picker('mobilenet')
for i in tqdm_nb(range(len(filenames))):
    feature_list.append(extract_features(filenames[i], model))



  0%|          | 0/1000 [00:00<?, ?it/s]

In [23]:
batch_size = 64
datagen = tensorflow.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)

generator = datagen.flow_from_directory(root_dir,
                                        target_size=(224, 224),
                                        batch_size=batch_size,
                                        class_mode=None,
                                        shuffle=False)

num_images = len(generator.filenames)

num_epochs = int(math.ceil(num_images / batch_size))

model = model_picker('mobilenet')
start_time = time.time()
feature_list = []
feature_list = model.predict(generator, num_epochs) # workers=4, use_multiprocessing=True)
end_time = time.time()

Found 8677 images belonging to 101 classes.


In [25]:
for i, features in enumerate(feature_list):
    feature_list[i] = features / norm(features)

feature_list = feature_list.reshape(num_images, -1)

print("Num images   = ", len(generator.classes))
print("Shape of feature_list = ", feature_list.shape)
print("Time taken in sec = ", end_time - start_time)

Num images   =  8677
Shape of feature_list =  (8677, 1024)
Time taken in sec =  278.1998007297516


In [37]:
filenames = [root_dir + '/' + s for s in generator.filenames]

In [38]:
!mkdir data

In [39]:
TRAIN_SAMPLES = 8677
NUM_CLASSES = 101
IMG_WIDTH, IMG_HEIGHT = 224, 224

In [40]:
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   zoom_range=0.2)

In [41]:
train_generator = train_datagen.flow_from_directory(root_dir,
                                                    target_size=(IMG_WIDTH,IMG_HEIGHT),
                                                    batch_size=batch_size,
                                                    shuffle=True,
                                                    seed=12345,
                                                    class_mode='categorical')

Found 8677 images belonging to 101 classes.


In [42]:
def model_maker():
    base_model = ResNet50(include_top=False, input_shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    for layer in base_model.layers[:]:
        # layer.trainable = False
        layer.trainable = True
    input = Input(shape=(IMG_WIDTH, IMG_HEIGHT, 3))
    custom_model = base_model(input)
    custom_model = GlobalAveragePooling2D()(custom_model)
    custom_model = Dense(64, activation='relu')(custom_model)
    custom_model = Dropout(0.5)(custom_model)
    predictions = Dense(NUM_CLASSES, activation='softmax')(custom_model)
    return Model(inputs=input, outputs=predictions)

In [44]:
from keras.models import load_model

In [ ]:
model_finetuned = model_maker()
model_finetuned.compile(loss='categorical_crossentropy', optimizer=tensorflow.keras.optimizers.Adam(0.001), metrics=['acc'])
# model_finetuned = load_model('./model-finetuned.h5')
model_finetuned.fit(train_generator, steps_per_epoch=math.ceil(float(TRAIN_SAMPLES) / batch_size), epochs=5)

Epoch 1/5
136/136 [==============================] - 5669s 42s/step - loss: 3.5238 - acc: 0.2856
Epoch 2/5
136/136 [==============================] - 5689s 42s/step - loss: 2.8429 - acc: 0.3869
Epoch 3/5
136/136 [==============================] - 5752s 42s/step - loss: 2.4456 - acc: 0.4513
Epoch 4/5
 56/136 [===========>..................] - ETA: 56:55 - loss: 2.1125 - acc: 0.5081